### Import libraries

In [ ]:
import polars as pl
import os
from deltalake import DeltaTable
from lakefs import Repository
from lakefs.client import Client
from dotenv import load_dotenv

load_dotenv()

### Configure lakefs client, storage options

In [ ]:
lakefs_endpoint = "http://127.0.0.1:8000"

lakefs_client = Client(
    host=lakefs_endpoint,
    username=os.environ["LAKEFS_ACCESS_KEY"],
    password=os.environ["LAKEFS_SECRET_ACCESS_KEY"],
)
storage_options={
        "access_key_id": os.environ["LAKEFS_ACCESS_KEY"], 
        "secret_access_key": os.environ["LAKEFS_SECRET_ACCESS_KEY"], 
        "endpoint":lakefs_endpoint,
        "allow_http": "true",
        }

bronze_repo = Repository("bronze", client=lakefs_client)
# silver_repo = Repository("silver", client=lakefs_client)
# gold_repo = Repository("gold", client=lakefs_client)

### Create lakefs branch

In [ ]:
bronze_repo.branch("feat-multi-table-transactions").create(source_reference="main", exist_ok=True)

### Write multiple tables

In [ ]:
first_table = pl.DataFrame({"foo": [1,2,3,4,5], "bar": ["delta-rs", "is", "pretty","awesome", "ey"]})
second_table = pl.DataFrame({"foo": [10,11,12,13,14], "bar": ["delta-rs", "is", "pretty","awesome", "ey"]})

first_table.write_delta(
    "lakefs://bronze/feat-multi-table-transactions/first_table", 
    storage_options=storage_options,
    mode='overwrite',
)
second_table.write_delta(
    "lakefs://bronze/feat-multi-table-transactions/second_table", 
    storage_options=storage_options,
    mode='overwrite'
)

### Merge as single lakefs commit into `main`

In [ ]:
bronze_repo.branch("feat-multi-table-transactions").merge_into("main", squash_merge=True, message="merging table 1 and table 2 as multi-table-transaction")

In [ ]:
bronze_repo.branch("main").revert(reference="94f92ebf3c7bfed1a5ff403885c61b8147f2ed1ba312c5e0e0c26197994bdfe2")